## Notebook to generate Master Mapping excell for Size (subsize resolved) vs. Category &   Channel & Region 
Jul 2020 by Rafal Rakowski

In [43]:
# Set environment
import pandas as pd
from pandas import Series,DataFrame
from googletrans import Translator
import numpy as np
import re
import sys

In [44]:
# Check path
%pwd

'/Users/Raf/ecommerce_practical/Cross_Mapping_Project'

In [45]:
# Read in with API the eBay UK_Item Specifics_January_2020.xlsx into Python Pandas
master_mapper = pd.ExcelFile('eBay UK_Item Specifics_January_2020.xlsx')
data_frame = master_mapper.parse('eBay UK')

In [46]:
# Show Python imported data frame of the eBay UK_Item Specifics_January_2020.xlsx table
data_frame

,Category ID,Category Name,Item Specific Name,Required?,Allows Variations?,eBay Values Only?,Option List
0,96762,Antiques > Antique Clocks > Bracket Clocks > P...,Clock Type,No,Yes,No,"Alarm Clock, Backward Clock, Bracket Clock, Ch..."
1,96762,Antiques > Antique Clocks > Bracket Clocks > P...,Age,No,No,No,"Antique, Post-1900, Antique, Pre-1900, Antique..."
2,96762,Antiques > Antique Clocks > Bracket Clocks > P...,Type,No,No,No,Bracket Clock
3,96762,Antiques > Antique Clocks > Bracket Clocks > P...,Style/ Origin/ Theme,No,No,No,"American, Art Deco, Chinese, Edwardian, French..."
4,96762,Antiques > Antique Clocks > Bracket Clocks > P...,Material,No,No,No,"Bakelite, Brass, Bronze, Cane/ Wicker, Ceramic..."
...,...,...,...,...,...,...,...
191562,90951,Wholesale & Job Lots > Tickets & Travel,Unit Type,No,No,YES,"100g, 100ml, 10g, 10ml, kg, L, m, m², m³, Unit"
191563,90951,Wholesale & Job Lots > Tickets & Travel,Brand,No,No,No,NaN
191564,26424,Wholesale & Job Lots > Toys & Games,Unit Quantity,No,No,No,NaN
191565,26424,Wholesale & Job Lots > Toys & Games,Unit Type,No,No,YES,"100g, 100ml, 10g, 10ml, kg, L, m, m², m³, Unit"


In [47]:
# Create an empty output dataframe (to populate with processed data and to export to excel the revised here Master
# Mapping) with the key size-mapper columns
df_output = pd.DataFrame(columns=['Category ID', 'Category Name', 'Item Specific Name', 'Option List']) 

In [48]:
# Cleanse data, remove empty records
df_output['Category ID'] = data_frame['Category ID']
df_output['Category Name'] = data_frame['Category Name']
df_output['Item Specific Name'] = data_frame['Item Specific Name']
df_output['Option List'] = data_frame['Option List']
for col in df_output.columns:
    bool_series = pd.isnull(df_output[col])  
    df_output[col][bool_series] = 'empty'

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [49]:
# Introduce some key names for products associated to a size feature
product_name = 'Clothes Clothing Shoes' 
product_name_list = product_name.split(' ')
product_name_list

['Clothes', 'Clothing', 'Shoes']

In [50]:
# Match either word in new product name to records in Category Name column 
# Pick indexes for found hits and insert corresponding to them rows to the created output data frame

bla = [False] * len(df_output['Category Name'])
result = np.array(bla)

for word in product_name_list:
    print(word)
    matched = df_output['Category Name'].str.contains(r'\b{}\b'.format(word)) 
    matched = np.array(matched)
    y=any(matched.tolist())
    #print(y)
    if  y:
        result = matched | result
hits = [i for i, x in enumerate(result) if x]
df_output_temp = df_output[result]

Clothes
Clothing
Shoes


In [51]:
# See how many lines there are in the output frame (it is ~11k rows from original ~ 190k)
df_output_temp['Category Name'].size

10901

In [52]:
# Match either phrase containing 'Size' word in 'Item Specific Name' column and insert matched rows to the output 
# dataframe. Pandas will align and remove previously insterted data rows automatically!

size_list = ['Size'] 
bla = [False] * len(df_output_temp['Item Specific Name'])
result = np.array(bla)

for word in size_list:
    print(word)
    matched = df_output_temp['Item Specific Name'].str.contains(r'\b{}\b'.format(word)) 
    matched = np.array(matched)
    y=any(matched.tolist())
    #print(y)
    if  y:
        result = matched | result
hits = [i for i, x in enumerate(result) if x]
df_output_sizes = df_output_temp[result]

Size


In [14]:
# Now there is even less data (left 780 rows!)
df_output_sizes['Category Name'].size

780

In [69]:
# Show output frame
df_output_sizes

,Category ID,Category Name,Item Specific Name,Option List
18285,109674,"Business, Office & Industrial > Facility Maint...",Shoe Size,"UK 4.5 (Men's), UK 5 (Men's), UK 5.5 (Men's), ..."
44871,260022,"Clothes, Shoes & Accessories > Baby > Baby & T...",Size,"0-3 Months, 12-18 Months, 12 Months, 18-24 Mon..."
44888,260020,"Clothes, Shoes & Accessories > Baby > Baby & T...",Size,"0-3 Months, 12-18 Months, 12 Months, 18-24 Mon..."
44898,260035,"Clothes, Shoes & Accessories > Baby > Baby & T...",Size,"0-3 Months, 12-18 Months, 12 Months, 18-24 Mon..."
44909,139762,"Clothes, Shoes & Accessories > Baby > Baby & T...",Size,"Newborn, 0-3 Months, 3-6 Months, 6-9 Months, 9..."
...,...,...,...,...
178202,177106,"Vehicle Parts & Accessories > Clothing, Helmet...",Size,"34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56..."
178220,177108,"Vehicle Parts & Accessories > Clothing, Helmet...",Size,"XXS, XS, S, M, L, XL, XXL, XXXL, One Size, Child"
178238,177109,"Vehicle Parts & Accessories > Clothing, Helmet...",Size,"34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56..."
178258,177110,"Vehicle Parts & Accessories > Clothing, Helmet...",Size,"34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56..."


In [16]:
# Repeat all done above for eBay DE_Items (above it was for eBay UK_Item)

# Read in with API the eBay DE_Item Specifics_January_2020.xlsx into Python Pandas
master_mapper_DE = pd.ExcelFile('eBay DE_Item Specifics_January 2020.xlsx')
data_frame_DE = master_mapper_DE.parse('eBay DE')

#Create an empty output dataframe with the key size-mapper columns
df_output_DE = pd.DataFrame(columns=['Category ID', 'Category Name', 'Item Specific Name', 'Option List'])
df_output_DE['Category ID'] = data_frame_DE['Category ID']
df_output_DE['Category Name'] = data_frame_DE['Category Name']
df_output_DE['Item Specific Name'] = data_frame_DE['Item Specific Name']
df_output_DE['Option List'] = data_frame_DE['Option List']
for col in df_output_DE.columns:
    bool_series = pd.isnull(df_output_DE[col])  
    df_output_DE[col][bool_series] = 'empty'
    
product_name_DE = 'Kleidung Bekleidung Kleider Schuhe' 
product_name_list_DE = product_name_DE.split(' ')

# Match either word in new product name to records in 'Category Name' column 
# Pick indexes for found hits and insert corresponding to them rows to the created output dataframe
bla = [False] * len(df_output_DE['Category Name'])
result = np.array(bla)

for word in product_name_list_DE:
    print(word)
    matched = df_output_DE['Category Name'].str.contains('{}'.format(word)) 
    matched = np.array(matched)
    y=any(matched.tolist())
    #print(y)
    if  y:
        result = matched | result
hits_DE = [i for i, x in enumerate(result) if x]

df_output_temp_DE = df_output_DE[result]

translator = Translator()
translations = translator.translate(['Size'], dest='de')
for translation in translations:
    term = translation.text
size_list_DE = [term] 

# Match either phrase containing 'Size' word in 'Item Specific Name' column and insert matched rows to output frame
# Py Pandas will align and remove previously insterted data rows automatically!
bla = [False] * len(df_output_temp_DE['Item Specific Name'])
result = np.array(bla)

for word in size_list_DE:
    print(word)
    matched = df_output_temp_DE['Item Specific Name'].str.contains('{}'.format(word)) 
    matched = np.array(matched)
    y=any(matched.tolist())
    #print(y)
    if  y:
        result = matched | result
hits = [i for i, x in enumerate(result) if x]
df_output_sizes_DE = df_output_temp_DE[result]

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


Kleidung
Bekleidung
Kleider
Schuhe
Größe


In [17]:
# Show the output frame for df_output_sizes_DE
df_output_sizes_DE

,Category ID,Category Name,Item Specific Name,Option List
15267,42508,Auto & Motorrad: Teile > Automobilia > Bekleid...,Größe,"XS, S, M, L, XL, XXL"
15397,73665,Auto & Motorrad: Teile > Auto-Motorsport > Bek...,Größe,"XS, S, M, L, XL, XXL, XXXL"
15413,73666,Auto & Motorrad: Teile > Auto-Motorsport > Bek...,Größe,"XS, S, M, L, XL, XXL, XXXL"
15429,73668,Auto & Motorrad: Teile > Auto-Motorsport > Bek...,Größe,"XS, S, M, L, XL, XXL, XXXL"
15445,73667,Auto & Motorrad: Teile > Auto-Motorsport > Bek...,Größe,"XS, S, M, L, XL, XXL, XXXL"
...,...,...,...,...
178065,159167,Sport > Weitere Sportarten > Turnen & Gymnasti...,Größe,"XS, S, M, L, XL, XXL"
178198,4939,Sport > Weitere Wintersportarten > Eislaufen >...,Größe,"XXS, XS, S, M, ML, L, XL, 2XL, 3XL, 4XL, 32 - ..."
178309,77617,Sport > Weiterer Wassersport > Kitesurfen > Be...,Größe,"XXS, XS, S, M, ML, L, XL, 2XL, 3XL, 4XL, 32 - ..."
178419,77624,Sport > Weiterer Wassersport > Wakeboarden > B...,Größe,"XXS, XS, S, M, ML, L, XL, 2XL, 3XL, 4XL, 32 - ..."


In [18]:
# Combine (merge) both tables 'eBay UK_Item' & 'eBay DE_Item' towards creating the Master MApping for Clothing Sizes
combined_mapper = pd.merge(df_output_sizes, df_output_sizes_DE, on = 'Category ID', how = 'outer', suffixes =('_UK', '_DE'))

In [28]:
# Show the combine table
combined_mapper

,Category ID,Category Name_UK,Item Specific Name_UK,Option List_UK,Category Name_DE,Item Specific Name_DE,Option List_DE
0,109674,"Business, Office & Industrial > Facility Maint...",Shoe Size,"UK 4.5 (Men's), UK 5 (Men's), UK 5.5 (Men's), ...",NaN,NaN,NaN
1,260022,"Clothes, Shoes & Accessories > Baby > Baby & T...",Size,"0-3 Months, 12-18 Months, 12 Months, 18-24 Mon...",Kleidung & Accessoires > Babys > Mode für Baby...,Größe,"80, 86, Bis Größe 50, Einheitsgröße, Größe 56,..."
2,260020,"Clothes, Shoes & Accessories > Baby > Baby & T...",Size,"0-3 Months, 12-18 Months, 12 Months, 18-24 Mon...",Kleidung & Accessoires > Babys > Mode für Baby...,Größe,"80, 86, Bis Größe 50, Einheitsgröße, Größe 56,..."
3,260035,"Clothes, Shoes & Accessories > Baby > Baby & T...",Size,"0-3 Months, 12-18 Months, 12 Months, 18-24 Mon...",NaN,NaN,NaN
4,139762,"Clothes, Shoes & Accessories > Baby > Baby & T...",Size,"Newborn, 0-3 Months, 3-6 Months, 6-9 Months, 9...",Kleidung & Accessoires > Babys > Mode für Baby...,Größe,"Bis Größe 50, Größe 56, Größe 62, Größe 68, Gr..."
...,...,...,...,...,...,...,...
988,159167,NaN,NaN,NaN,Sport > Weitere Sportarten > Turnen & Gymnasti...,Größe,"XS, S, M, L, XL, XXL"
989,4939,NaN,NaN,NaN,Sport > Weitere Wintersportarten > Eislaufen >...,Größe,"XXS, XS, S, M, ML, L, XL, 2XL, 3XL, 4XL, 32 - ..."
990,77617,NaN,NaN,NaN,Sport > Weiterer Wassersport > Kitesurfen > Be...,Größe,"XXS, XS, S, M, ML, L, XL, 2XL, 3XL, 4XL, 32 - ..."
991,77624,NaN,NaN,NaN,Sport > Weiterer Wassersport > Wakeboarden > B...,Größe,"XXS, XS, S, M, ML, L, XL, 2XL, 3XL, 4XL, 32 - ..."


In [19]:
# Save the combined table to an excel file (it could be saved as a sub_spreadsheet of existing Master Mapping excel)
combined_mapper.to_excel('Item Specific Size Mapper.xlsx', header = True)

In [15]:
# Translate test to be implemented later on for non_English regions
translator = Translator()
translations = translator.translate(['Size'], dest='de')
for translation in translations:
    word = translation.text

In [16]:
word

'Größe'

## Notebook to map a new product (by its name) to Master Mapping excel and also to generate a table with aligned paths (for different regions and channels) of Product Categories & IDs per Channel&Region. Input product name to be located in the first column.
Jul 2020 by Rafal Rakowski

In [102]:
# Set compute environment

import pandas as pd
from pandas import Series,DataFrame
from googletrans import Translator
import numpy as np
import re
from collections import OrderedDict

In [103]:
%pwd

'/Users/Raf/ecommerce_practical/Cross_Mapping_Project'

In [104]:
# Read in with API the Master_Mapping.xlsx into Python Pandas
master_mapper = pd.ExcelFile('Master_Mapping.xlsx')
data_frame = master_mapper.parse('Sheet1')

In [105]:
# Create empty output dataframe for the mapping of new products
df_output = pd.DataFrame(columns=data_frame.columns) 
count = 0 
df_output.insert(0, 'Product Name', [])
df_output 

,Product Name,ID,Master Path,Amazon UK ID,Amazon UK Path,Amazon DE ID,Amazon DE Path,Amazon FR ID,Amazon FR Path,Amazon IT ID,...,Ebay ES Path,Ebay US,Ebay US Path,Ebay CA ID,Ebay CA Path,Ebay AU ID,Ebay AU Path,Test Channel,Test Path,Unnamed: 36


In [106]:
# Show a few lines of Master spreadsheet
data_frame

,ID,Master Path,Amazon UK ID,Amazon UK Path,Amazon DE ID,Amazon DE Path,Amazon FR ID,Amazon FR Path,Amazon IT ID,Amazon IT Path,...,Ebay ES Path,Ebay US,Ebay US Path,Ebay CA ID,Ebay CA Path,Ebay AU ID,Ebay AU Path,Test Channel,Test Path,Unnamed: 36
0,1,/Categories/3D Printing & Scanning/3D Printer ...,6.283024e+09,/Categories/3D Printing & Scanning/3D Printer ...,6.589298e+09,Elektronik & Foto/Computer & Zubehör/Computer-...,6.606606e+09,Informatique/Accessoires/Composants pour impri...,6.572429e+09,Informatica/Accessori/Ricambi e accessori per ...,...,Informática y tablets > Impresión 3D > Impreso...,260315.0,Computers/Tablets & Networking > 3D Printers &...,183066.0,Computers/Tablets & Networking > 3D Printers &...,183066,Computers/Tablets & Networking > 3D Printers &...,12345.0,This isnt working,NaN
1,2,/Categories/3D Printing & Scanning/3D Printer ...,6.283025e+09,/Categories/3D Printing & Scanning/3D Printer ...,6.589297e+09,Elektronik & Foto/Computer & Zubehör/Computer-...,6.606607e+09,Informatique/Accessoires/Composants pour impri...,6.572430e+09,Informatica/Accessori/Ricambi e accessori per ...,...,Informática y tablets > Impresión 3D > Impreso...,183066.0,Computers/Tablets & Networking > 3D Printers &...,183066.0,Computers/Tablets & Networking > 3D Printers &...,183066,Computers/Tablets & Networking > 3D Printers &...,NaN,NaN,NaN
2,3,/Categories/3D Printing & Scanning/3D Printer ...,6.283027e+09,/Categories/3D Printing & Scanning/3D Printer ...,6.589296e+09,Elektronik & Foto/Computer & Zubehör/Computer-...,6.606609e+09,Informatique/Accessoires/Composants pour impri...,6.572432e+09,Informatica/Accessori/Ricambi e accessori per ...,...,Informática y tablets > Impresión 3D > Impreso...,183066.0,Computers/Tablets & Networking > 3D Printers &...,183066.0,Computers/Tablets & Networking > 3D Printers &...,183066,Computers/Tablets & Networking > 3D Printers &...,NaN,NaN,NaN
3,4,/Categories/3D Printing & Scanning/3D Printer ...,8.624609e+09,/Categories/3D Printing & Scanning/3D Printer ...,8.623223e+09,Elektronik & Foto/Computer & Zubehör/Computer-...,8.641317e+09,Informatique/Accessoires/Composants pour impri...,8.641300e+09,Informatica/Accessori/Ricambi e accessori per ...,...,Informática y tablets > Impresión 3D > Impreso...,183066.0,Computers/Tablets & Networking > 3D Printers &...,183066.0,Computers/Tablets & Networking > 3D Printers &...,183066,Computers/Tablets & Networking > 3D Printers &...,NaN,NaN,NaN
4,5,/Categories/3D Printing & Scanning/3D Printer ...,6.283026e+09,/Categories/3D Printing & Scanning/3D Printer ...,6.589293e+09,Elektronik & Foto/Computer & Zubehör/Computer-...,6.606608e+09,Informatique/Accessoires/Composants pour impri...,6.572431e+09,Informatica/Accessori/Ricambi e accessori per ...,...,Informática y tablets > Impresión 3D > Impreso...,183066.0,Computers/Tablets & Networking > 3D Printers &...,183066.0,Computers/Tablets & Networking > 3D Printers &...,183066,Computers/Tablets & Networking > 3D Printers &...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48464,48465,Watches/Men/Wrist Watches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48465,48466,Watches/Women/Pocket Watches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48466,48467,Watches/Women/Watch Bands,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48467,48468,Watches/Women/Wrist Watches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Show data of Master spreadsheet
data_frame.info()

In [107]:
master_path=data_frame['Master Path']
master_path[0]

'/Categories/3D Printing & Scanning/3D Printer Parts & Accessories/3D Printer Accessories'

In [108]:
path_list=master_path[0].split('/')
path_list

['',
 'Categories',
 '3D Printing & Scanning',
 '3D Printer Parts & Accessories',
 '3D Printer Accessories']

In [109]:
path_list[-1]

'3D Printer Accessories'

In [110]:
# Save split name keyword from the outermost level of 'Master Path'
var = []
for category in data_frame['Master Path'].str.split('/')[:] :  
    var.append(category[-1])  
data_frame['Master_list'] = var
var

['3D Printer Accessories',
 '3D Printer Controllers',
 '3D Printer Driver Modules',
 '3D Printer Extruder Parts',
 '3D Printer Extruders',
 '3D Printer Interface Modules',
 '3D Printer Motors',
 '3D Printer Platforms',
 '3D Printer Timing Belts',
 '3D Printers',
 'Filament 3D Printing Materials',
 'Granular 3D Printing Materials',
 'Liquid 3D Printing Materials',
 '3D Scanners',
 'Dressing Tools',
 'Mandrels',
 'Wheel Adapters',
 'Wheel Flanges',
 'Abrasive Belts',
 'Bevel Brushes',
 'Cup Brushes',
 'Disc Brushes',
 'End Brushes',
 'Power Brush Sets',
 'Spiral Brushes',
 'Strip Brushes',
 'Tube Brushes',
 'Wheel Brushes',
 'Scratch Brushes',
 'Bands',
 'Cartridge Rolls',
 'Grinding Cones',
 'Grinding Mounted Points',
 'Grinding Plugs',
 'Non-woven Bits',
 'Angle Grinder Wheels',
 'Bench Grinding Wheels',
 'Bristle Discs',
 'Convolute Wheels',
 'Cup Wheels',
 'Cut-off Wheels',
 'Die Grinder Wheels',
 'Fladder Blades',
 'Flap Discs',
 'Flap Wheels',
 'Grinding Discs',
 'Grinding Points',

In [111]:
data_frame[9102:9105]

,ID,Master Path,Amazon UK ID,Amazon UK Path,Amazon DE ID,Amazon DE Path,Amazon FR ID,Amazon FR Path,Amazon IT ID,Amazon IT Path,...,Ebay US,Ebay US Path,Ebay CA ID,Ebay CA Path,Ebay AU ID,Ebay AU Path,Test Channel,Test Path,Unnamed: 36,Master_list
9102,9103,/Categories/Nappy Changing/Nappies/Disposable ...,9.641487e+09,/Categories/Nappy Changing/Nappies/Disposable ...,9.645592e+09,Baby/Wickeln/Windeln/Einwegwindeln/Babywindeln,9.644690e+09,Bébé et Puériculture/Change de Bébé/Couches/Co...,9.645297e+09,Prima infanzia/Cambio del pannolino/Pannolini/...,...,26268.0,NaN,20399.0,Baby > Diapering > Other Diapering,20399,Baby > Nappies > Other Nappies,NaN,Baby > Nappies > Other Nappies,NaN,Baby Nappies
9103,9104,/Categories/Nappy Changing/Nappies/Disposable ...,1.890286e+09,/Categories/Nappy Changing/Nappies/Disposable ...,3.969105e+09,Baby/Wickeln/Windeln/Einwegwindeln/Schwimmwindeln,3.966227e+09,Bébé et Puériculture/Change de Bébé/Couches/Co...,1.806559e+09,Prima infanzia/Cambio del pannolino/Pannolini/...,...,26268.0,NaN,100222.0,Baby > Diapering > Swim Diapers,20399,Baby > Nappies > Other Nappies,NaN,Baby > Nappies > Other Nappies,NaN,Disposable Swim Nappies
9104,9105,/Categories/Nappy Changing/Nappies/Disposable ...,1.890264e+09,/Categories/Nappy Changing/Nappies/Disposable ...,3.969107e+09,Baby/Töpfchen & Trittschemel/Über- & Spreizhosen,3.966228e+09,Bébé et Puériculture/Apprentissage de la propr...,1.806560e+09,Prima infanzia/Cambio del pannolino/Pannolini/...,...,37631.0,Baby > Potty Training,37631.0,Baby > Potty Training,15559,Baby > Nappies > Nappies (Disposable),NaN,Baby > Nappies > Nappies (Disposable),NaN,Disposable Training Pants


In [123]:
# Input (using a mini user interface) exemplary products by keyword names to match to Master Mapping in the next cell

product_name = input('Type product name (example: Terminal Cables)')
#product_name = 'Terminal Cables' #  Terminal Cables  blue Disposable, Training pants
product_name = str(product_name)
product_name_list = product_name.split(' ')
product_name_list

Type product name (example: Terminal Cables)D-Terminal Cables


['D-Terminal', 'Cables']

In [127]:
# Script matching either word in a new product name string to select the best fitting row in the 'Master' table. 
# If there is more good matches then using another mini GUI one can select a correct row/ID from the suggested 
# by the algorithm rows

# Initialize some boolean series
bla_M = [True] * len(data_frame['Master_list'])
result_M = np.array(bla_M)

bla_m = [False] * len(data_frame['Master_list'])
result_m = np.array(bla_m)

# Iterate through all words in the new product name to match highest level category path phrases containing 
# these words
for word in product_name_list:
    print(word)
    matched_M = data_frame['Master_list'].str.contains(r'\b{}\b'.format(word)) 
    matched_M = np.array(matched_M)
    matched_m = matched_M
    y=any(matched_M.tolist())
    print(y)
    if  y:
        result_M = matched_M & result_M
    result_m = matched_m | result_m
y=any(result_M.tolist()) 
print(y)
if y:    
    result_M = result_M.tolist()
else: 
    result_M = result_m.tolist()
    
# Pick the matching lines' IDs and depending on their number break (for none), continue for a single match 
# or throw options on the screen to choose from (for multiple matches)
hits_Master = [i for i, x in enumerate(result_M) if x]

if      len(hits_Master) == 0: 
        print('+++++++++ There is no record yet for this product name in the system ++++++++++')
        #Room here to elaborate new products inputs to Master Mapper
        hits_Master = []
elif    len(hits_Master) >1:
        print('\nThese are possible Master Mapper matches:\n')
        for i in hits_Master:
            print(i+1)
            print(var[i])
            print(data_frame['Master Path'][i])
            print()
        prompt_feedback = input('Input in the box the right ID number()')
        hits_M = int(prompt_feedback)
        hits_M  -= 1
    
else:    
        hits_M = int(str(hits_Master[0]))
        hits_M += 1 
print('Master_ID(s) = '+str(hits_M))

print('\n%%%%%%%%%%%% below there are the matches for channels & regions %%%%%%%%%%%\n')

# Do the same as above (for Master Category Path') for all the other regions and channels (simple rewriting won't 
# work since the lines in different columns are misaligned - they can be vertically shifted by a row or two)

# Pick data from the original df for the output dataframe
dictionary = dict.fromkeys(data_frame.columns) 
dictionary['ID'] = data_frame['ID'][hits_Master[0]]
dictionary['Master Path'] = data_frame['Master Path'][hits_Master[0]]
t = 0

# Some cleansing function to automate in the main matching loop
def function(data_frame, k):
    bool_series = pd.isnull(data_frame[k])  
    data_frame[bool_series] = 'emptycell'
    var2 = []
    for category in data_frame[k].str.split('/')[:] : 
        var2.append(category[-1]) 
    return var2
counter = 2

# Iterate matching through all channels and regions
for k in data_frame.columns[3:34:2]: # for tests it was [3:4]
    var2 = function(data_frame, k)
    data_frame[k + ' list'] = var2
    
# About below temporary assignments: I substitue (before I add robust translation part) other than UK region data with the UK data since algorithm due to the language mismatch 
# won't find any table data matches. In the SIZE MASTER MAPPER Python code it is showcased how to go around using a
# translation automatically
    if   k == 'Amazon ES Path': 
         k = 'Amazon UK Path'
         t = 1
    elif k ==  'Ebay DE Path':
         k = 'Ebay UK Path'
         t = 2 
    elif k ==  'Amazon CAN Path':
         k =  'Amazon UK Path'
         t = 3   
    elif k ==  'Ebay FR Path': 
         k =  'Ebay UK Path'
         t = 4     
    elif k ==  'Ebay IT Path': 
         k =  'Ebay UK Path'
         t = 5  
    elif k ==  'Amazon IT Path':
         k =  'Amazon UK Path'
         t = 6   
    elif k ==  'Amazon FR Path':
         k =  'Amazon UK Path'
         t = 7  
    bla = [False] * len(data_frame[k + ' list'] )
    result = np.array(bla)

    for word in product_name_list:
        print(word)
        matched = data_frame[k + ' list'] .str.contains(r'\b{}\b'.format(word)) 
        matched = np.array(matched)
        y=any(matched.tolist())
        if  y:
            result = matched & result
        y=any(result)
        print(y)
# Some extra tests not to miss a possible match if there is no one in the first try
        if y == False:
            matched = data_frame[k].str.contains('{}'.format(word)) 
            matched = np.array(matched)
            result = matched & result
            y=any(matched.tolist()) 
            print(y)      
    hits = ([i for i, x in enumerate(result) if x])
    print(k)
    print([i + 1 for i, x in enumerate(result) if x])
    print() 
    
    if hits == []:
        for word in product_name_list:
            matched = data_frame[k + ' list'].str.contains(r'\b{}\b'.format(word)) 
       
            matched = np.array(matched)
            y=any(matched.tolist())

            if  y:
                result = matched | result
            y=any(result.tolist())
      
        result = result.tolist()
        
    global_hits = [i for i, x in enumerate(result) if x]
    if  global_hits == []:break
    else:
        #for i in global_hits:
            #print(i+1)
            #print(var2[i])
            #print(data_frame[k + 'list'][i])                         
        hits = min(global_hits, key=lambda x:abs(x - (hits_M - 1)))
        print(hits)   
        if   t == 1:
             k = 'Amazon ES Path'
        elif t == 2:
             k = 'Ebay DE Path'
        elif t == 3:
             k = 'Amazon CAN Path'
        elif t == 4:
             k = 'Ebay FR Path'        
        elif t == 5:
             k = 'Ebay IT Path' 
        elif t == 6:
             k = 'Amazon IT Path'        
        elif t == 7:
             k = 'Amazon FR Path' 
# Save the matched  Product Name  data rows to output dataframe: df_output
        dictionary[k] = data_frame[k][hits]
        dictionary[data_frame.columns[counter]] =  data_frame[data_frame.columns[counter]][hits]
        counter += 2
        t = 0
df_output = df_output.append(dictionary, ignore_index=True)
df_output['Product Name'] [count] = product_name
count += 1

D-Terminal
True
Cables
True
True
Master_ID(s) = 131

%%%%%%%%%%%% below there are the matches for channels & regions %%%%%%%%%%%

D-Terminal
False
True
Cables
False
True
Amazon UK Path
[]

129
D-Terminal
False
True
Cables
False
False
Amazon DE Path
[]

130
D-Terminal
False
True
Cables
False
True
Amazon UK Path
[]

129
D-Terminal
False
True
Cables
False
True
Amazon UK Path
[]

129
D-Terminal
False
True
Cables
False
True
Amazon UK Path
[]

129
D-Terminal
False
True
Cables
False
True
Amazon US Path
[]

130
D-Terminal
False
True
Cables
False
True
Amazon UK Path
[]

129
D-Terminal
False
True
Cables
False
True
Amazon AU Path
[]

130
D-Terminal
False
False
Cables
False
True
Ebay UK Path
[]

129
D-Terminal
False
False
Cables
False
True
Ebay UK Path
[]

129
D-Terminal
False
False
Cables
False
True
Ebay UK Path
[]

129
D-Terminal
False
False
Cables
False
True
Ebay UK Path
[]

129
D-Terminal
False
False
Cables
False
True
Ebay ES Path
[]

129
D-Terminal
False
False
Cables
False
True
Ebay US Path
[

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:165: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [125]:
# Show the df_output frame with an entry using 'mini' GUI for product name: D-Terminal Cables
Master_mapper = df_output 
df_output  

,Product Name,ID,Master Path,Amazon UK ID,Amazon UK Path,Amazon DE ID,Amazon DE Path,Amazon FR ID,Amazon FR Path,Amazon IT ID,...,Amazon IT Path list,Amazon US Path list,Ebay AU Path list,Ebay CA Path list,Ebay DE Path list,Ebay ES Path list,Ebay FR Path list,Ebay IT Path list,Ebay UK Path list,Ebay US Path list
0,Training pants,350,/Categories/American Football/Clothing/Boys/Tr...,1.730691e+09,/Categories/Baby/Baby Boys 0-24m/Underwear/Tra...,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,blue Disposable Training pants,9105,/Categories/Nappy Changing/Nappies/Disposable ...,1.890264e+09,/Categories/Nappy Changing/Nappies/Disposable ...,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Terminal Cables,131,/Categories/Accessories/Home Audio & Video Acc...,5.286000e+08,/Categories/Accessories/Home Audio & Video Acc...,1.3827e+09,Elektronik & Foto/Fernseher & Heimkino/Zubehör...,1.62516e+07,High-Tech/Consommables et Accessoires/Accessoi...,4.73318e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,D-Terminal Cables,131,/Categories/Accessories/Home Audio & Video Acc...,5.286000e+08,/Categories/Accessories/Home Audio & Video Acc...,1.3827e+09,Elektronik & Foto/Fernseher & Heimkino/Zubehör...,1.62516e+07,High-Tech/Consommables et Accessoires/Accessoi...,4.73318e+08,...,None,None,None,None,None,None,None,None,None,None


In [ ]:
# Save df_output table to an excel file (optionally can be saved as a sub_spreadsheet of existing Master Mapping excel)
Master_mapper.to_excel('Master Mapper.xlsx', header = True)